In [3]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import keras
from keras.layers import Dense, LSTM, SimpleRNN, GRU, Input
from keras.models import Model, Sequential  

df_train = pd.read_csv("./storage/Kaggle_M5_Luke/sales_train_validation.csv")
df_sell = pd.read_csv("./storage/Kaggle_M5_Luke/sell_prices.csv")
df_calendar = pd.read_csv("./storage/Kaggle_M5_Luke/calendar.csv")
df_ss = pd.read_csv("./storage/Kaggle_M5_Luke/sample_submission_m5.csv")


Using TensorFlow backend.


In [4]:
days = range(1,1913+1) 
time_series_cols = [] 
for i in days: 
    time_series_cols.append('d_'+str(i)) 
time_series_cols[:10]

['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10']

In [5]:
time_series_data = df_train[time_series_cols] 
time_series_data

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [6]:
time_series_data = np.asarray(time_series_data) 
print(time_series_data.shape)

(30490, 1913)


In [47]:
# x_train: d_i to d_(i+6), y_train: d_(i+1) to d_(i+7), if time_steps = 7 
def split_xy1(dataset,time_steps): 
    x,y = list(), list() 
    for i in range(0,dataset.shape[1]-time_steps): 
        x.append(dataset[:,i:i+time_steps]) 
        y.append(dataset[:,i+1:i+time_steps+1]) 
    return np.array(x),np.array(y)   

x_train, y_train = split_xy1(time_series_data,7)
x_train = x_train.reshape(1906,7,30490) 
y_train = y_train.reshape(1906,7,30490) 
x_pred = time_series_data[:,time_series_data.shape[1]-7:] 
x_pred = x_pred.reshape(1,7,time_series_data.shape[0]) 
print(x_train.shape) 
print(y_train.shape)
print(x_pred.shape)

(1906, 7, 30490)
(1906, 7, 30490)
(1, 7, 30490)


In [55]:
def build_model(time_steps,features): 
    model = Sequential() 
    model.add(LSTM(8,input_shape=(time_steps,features),activation='relu',return_sequences=True)) 
    model.add(Dense(30490)) 
    return model 
model = build_model(7,30490) 
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_7 (LSTM)                (None, 7, 8)              975968    
_________________________________________________________________
dense_6 (Dense)              (None, 7, 30490)          274410    
Total params: 1,250,378
Trainable params: 1,250,378
Non-trainable params: 0
_________________________________________________________________


In [57]:
model.compile(loss='mse',optimizer='adam',metrics=['mse'])
model.fit(x_train,y_train,epochs=50) 

Epoch 1/50
1906/1906 [==============================] - 6s 3ms/step - loss: 13.3496 - mse: 13.3496
Epoch 2/50
1906/1906 [==============================] - 6s 3ms/step - loss: 13.1478 - mse: 13.1478
Epoch 3/50
1906/1906 [==============================] - 5s 3ms/step - loss: 13.1116 - mse: 13.1116
Epoch 4/50
1906/1906 [==============================] - 5s 3ms/step - loss: 13.0816 - mse: 13.0816
Epoch 5/50
1906/1906 [==============================] - 5s 3ms/step - loss: 13.0494 - mse: 13.0494
Epoch 6/50
1906/1906 [==============================] - 5s 3ms/step - loss: 13.0153 - mse: 13.0153
Epoch 7/50
1906/1906 [==============================] - 6s 3ms/step - loss: 12.9789 - mse: 12.9789
Epoch 8/50
1906/1906 [==============================] - 6s 3ms/step - loss: 12.9436 - mse: 12.9436
Epoch 9/50
1906/1906 [==============================] - 6s 3ms/step - loss: 12.9065 - mse: 12.9065
Epoch 10/50
1906/1906 [==============================] - 6s 3ms/step - loss: 12.8668 - mse: 12.8668
Epoch 11/

In [13]:
x_train = time_series_data[:,:-1] 
y_train = time_series_data[:,-1] 
x_pred = time_series_data[:,1:] 

print(x_train.shape) 
print(y_train.shape)
print(x_pred.shape)  

x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1) 
x_pred = x_pred.reshape(x_pred.shape[0],x_pred.shape[1],1) 

print(x_train.shape) 
print(x_pred.shape)

(30490, 1912)
(30490,)
(30490, 1912)
(30490, 1912, 1)
(30490, 1912, 1)


In [21]:
def build_model(cols): 
    model = Sequential() 
    model.add(LSTM(8,input_shape=(cols,1),activation='relu'))#,return_sequences=True)) 
    #model.add(LSTM(4))  
    model.add(Dense(1))  
    return model 
model = build_model(1912) 
model.summary()

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 8)                 320       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 9         
Total params: 329
Trainable params: 329
Non-trainable params: 0
_________________________________________________________________


In [29]:
predictions = [] 
for i in range(1,29): 
    # prepare data  
    dataset = time_series_data[:,::i] 
    x_train = dataset[:  3,:-1] 
    y_train = dataset[:3,-1] 
    x_pred = dataset[:3,1:] 
    print(x_train.shape) 
    print(y_train.shape) 
    print(x_pred.shape)
    x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1) 
    x_pred = x_pred.reshape(x_pred.shape[0],x_pred.shape[1],1) 
    model = build_model(x_train.shape[1]) 
    model.compile(loss='mse',optimizer='adam',metrics=['mse']) 
    model.fit(x_train,y_train,epochs=2)
    pred = model.predict(x_pred) 
    predictions.append(pred) 



(3, 1912)
(3,)
(3, 1912)
Epoch 1/2
3/3 [==============================] - 2s 647ms/step - loss: 0.6335 - mse: 0.6335
Epoch 2/2
3/3 [==============================] - 2s 561ms/step - loss: 0.6247 - mse: 0.6247
(3, 956)
(3,)
(3, 956)
Epoch 1/2
3/3 [==============================] - 1s 405ms/step - loss: 0.4097 - mse: 0.4097
Epoch 2/2
3/3 [==============================] - 1s 271ms/step - loss: 0.3989 - mse: 0.3989
(3, 637)
(3,)
(3, 637)
Epoch 1/2
3/3 [==============================] - 1s 314ms/step - loss: 0.4772 - mse: 0.4772
Epoch 2/2
3/3 [==============================] - 1s 189ms/step - loss: 0.4734 - mse: 0.4734
(3, 478)
(3,)
(3, 478)
Epoch 1/2
3/3 [==============================] - 1s 246ms/step - loss: 0.8981 - mse: 0.8981
Epoch 2/2
3/3 [==============================] - 0s 137ms/step - loss: 0.8873 - mse: 0.8873
(3, 382)
(3,)
(3, 382)
Epoch 1/2
3/3 [==============================] - 1s 216ms/step - loss: 0.1808 - mse: 0.1808
Epoch 2/2
3/3 [==============================] - 0s 107

In [9]:
full_pred = [] 
for i in range(len(time_series_data)):  
    print("Processing item {} .... ".format(i)) 
    predictions = [] 
    for j in range(1,29): 
        dataset = time_series_data[i,::j]  
        x_train, y_train = split_xy(dataset,3) 
        x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
        model = build_model(3) 
        model.compile(loss='mse',optimizer='adam',metrics=['mse']) 
        model.fit(x_train, y_train, epochs=3, verbose=1) 
        x_pred = dataset[-3:]  
        x_pred = x_pred.reshape(1,3,1) 
        pred = model.predict(x_pred) 
        predictions.append(pred[0][0]) 
    full_pred.append(predictions) 

Processing item 0 .... 
Epoch 1/3
1910/1910 [==============================] - 1s 639us/step - loss: 0.5105 - mse: 0.5105
Epoch 2/3
1910/1910 [==============================] - 1s 295us/step - loss: 0.4509 - mse: 0.4509
Epoch 3/3
1910/1910 [==============================] - 1s 328us/step - loss: 0.4391 - mse: 0.4391
Epoch 1/3
954/954 [==============================] - 1s 1ms/step - loss: 0.5004 - mse: 0.5004
Epoch 2/3
954/954 [==============================] - 0s 302us/step - loss: 0.4510 - mse: 0.4510
Epoch 3/3
954/954 [==============================] - 0s 285us/step - loss: 0.4209 - mse: 0.4209
Epoch 1/3
635/635 [==============================] - 1s 1ms/step - loss: 0.5528 - mse: 0.5528
Epoch 2/3
635/635 [==============================] - 0s 293us/step - loss: 0.5203 - mse: 0.5203
Epoch 3/3
635/635 [==============================] - 0s 347us/step - loss: 0.4959 - mse: 0.4959
Epoch 1/3
476/476 [==============================] - 1s 2ms/step - loss: 0.4992 - mse: 0.4992
Epoch 2/3
476/47

KeyboardInterrupt: 